<a href="https://www.kaggle.com/code/vijays140291/oscillation-prediction?scriptVersionId=194861934" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [21]:
#importing library
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [22]:
file1='/kaggle/input/oscillilation1212/Valeur_Flottante_QV1.csv'
file2='/kaggle/input/oscillilation1212/Valeur_Flottante_QV2.csv'
file3='/kaggle/input/oscillilation1212/Valeur_Flottante_QV3.csv'
file4='/kaggle/input/oscillilation1212/Valeur_Flottante_QV4.csv'
file5='/kaggle/input/oscillilation1212/Valeur_Flottante_QV5.csv'
file6='/kaggle/input/oscillilation1212/Valeur_Flottante_QVP.csv'




df=pd.read_csv(file1)
df

,Cycle Cumulé,Valeur suivante,Valeur flottante
0,0,2,2
1,2,9,11
2,11,-13,-4
3,-2,4,-9
4,2,12,16
...,...,...,...
2526,3,-11,-7
2527,-8,26,15
2528,18,-22,4
2529,-4,-3,-25


In [23]:
y=df['Valeur suivante']
x=df.drop('Valeur suivante',axis=1)

# Before Adding Features

In [24]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.9,random_state=122)
scaler=StandardScaler()
scaler.fit(x_train)
x_train=pd.DataFrame(scaler.transform(x_train),columns=x_train.columns,index=x_train.index)
x_test=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns,index=x_test.index)

# Before hypertuning

In [25]:
model=RandomForestRegressor()
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.6643504545450357

# Result

# After hypertuning

In [26]:
model = RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=100)
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.7171274700394132

In [27]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor



model = RandomForestRegressor()

param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [None, 10, 20],    # Maximum depth of each tree
    'min_samples_split': [2, 5, 10]  # Minimum samples required to split a node
}


grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')


grid_search.fit(x_train, y_train)


best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)


best_model = grid_search.best_estimator_
test_score = best_model.score(x_test, y_test)
print(f"Test Score (R^2): {test_score:.4f}")


Best Hyperparameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 100}
Test Score (R^2): 0.7153


# After adding new Features

In [28]:
df_new=df.copy()
df_new['Cycle greater or less']=df_new['Cycle Cumulé'].apply(lambda x:1 if x>0 else 0)
df_new['Valeur greater or less']=df_new['Valeur flottante'].apply(lambda x:1 if x>0 else 0)
df_new

,Cycle Cumulé,Valeur suivante,Valeur flottante,Cycle greater or less,Valeur greater or less
0,0,2,2,0,1
1,2,9,11,1,1
2,11,-13,-4,1,0
3,-2,4,-9,0,0
4,2,12,16,1,1
...,...,...,...,...,...
2526,3,-11,-7,1,0
2527,-8,26,15,0,1
2528,18,-22,4,1,1
2529,-4,-3,-25,0,0


In [29]:

df_new['Cycle_Moving Average'] = df_new['Cycle Cumulé'].rolling(window=2).mean()

df_new['Cycle_Cumulative Deviation'] = (df_new['Cycle Cumulé'] - df_new['Cycle Cumulé'].mean()).cumsum()
df_new['Valeur_Moving Average'] = df_new['Valeur flottante'].rolling(window=2).mean()

df_new['Valeur_Cumulative Deviation'] = (df_new['Valeur flottante'] - df_new['Valeur flottante'].mean()).cumsum()


In [30]:
df_new

,Cycle Cumulé,Valeur suivante,Valeur flottante,Cycle greater or less,Valeur greater or less,Cycle_Moving Average,Cycle_Cumulative Deviation,Valeur_Moving Average,Valeur_Cumulative Deviation
0,0,2,2,0,1,NaN,-3.046227e+00,NaN,2.001976e+00
1,2,9,11,1,1,1.0,-4.092454e+00,6.5,1.300395e+01
2,11,-13,-4,1,0,6.5,3.861320e+00,3.5,9.005927e+00
3,-2,4,-9,0,0,4.5,-1.184907e+00,-6.5,7.902015e-03
4,2,12,16,1,1,0.0,-2.231134e+00,3.5,1.600988e+01
...,...,...,...,...,...,...,...,...,...
2526,3,-11,-7,1,0,1.0,1.318491e+01,-2.0,5.992098e+00
2527,-8,26,15,0,1,-2.5,2.138680e+00,4.0,2.099407e+01
2528,18,-22,4,1,1,5.0,1.709245e+01,9.5,2.499605e+01
2529,-4,-3,-25,0,0,7.0,1.004623e+01,-10.5,-1.975504e-03


In [31]:
df_new.columns

Index(['Cycle Cumulé', 'Valeur suivante', 'Valeur flottante',
       'Cycle greater or less', 'Valeur greater or less',
       'Cycle_Moving Average', 'Cycle_Cumulative Deviation',
       'Valeur_Moving Average', 'Valeur_Cumulative Deviation'],
      dtype='object')

In [34]:
df_new['Cycle_Moving Average']=df_new['Cycle_Moving Average'].fillna(df_new['Cycle_Moving Average'].mean())

In [35]:
df_new['Valeur_Moving Average']=df_new['Valeur_Moving Average'].fillna(df_new['Valeur_Moving Average'].mean())

In [41]:
df_train=df_new.loc[1:2500]
df_test=df_new.loc[2500:]

In [42]:
df_test

,Cycle Cumulé,Valeur suivante,Valeur flottante,Cycle greater or less,Valeur greater or less,Cycle_Moving Average,Cycle_Cumulative Deviation,Valeur_Moving Average,Valeur_Cumulative Deviation
2500,22,-26,0,1,0,9.0,-3.661320e+01,5.5,2.694073e+01
2501,-4,0,-26,0,0,9.0,-4.365942e+01,-13.0,9.427104e-01
2502,0,1,1,0,1,-2.0,-4.670565e+01,-12.5,1.944686e+00
2503,1,-1,0,1,0,0.5,-4.875188e+01,0.5,1.946661e+00
2504,0,22,21,0,1,0.5,-5.179810e+01,10.5,2.294864e+01
2505,22,-21,1,1,1,11.0,-3.284433e+01,11.0,2.395061e+01
2506,1,-3,-24,1,0,11.5,-3.489056e+01,-11.5,-4.741209e-02
2507,-2,3,0,0,0,-0.5,-3.993678e+01,-12.0,-4.543659e-02
2508,1,15,18,1,1,-0.5,-4.198301e+01,9.0,1.795654e+01
2509,16,-13,2,1,1,8.5,-2.902924e+01,10.0,1.995851e+01


In [ ]:
df_test.reset_index()

In [ ]:
#df_test

In [44]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Example data (5 columns)
data = np.array(df_new)

# Prepare sequences (windowed sequences of input data)
def create_sequences(data, seq_length, target_column):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Use all columns in the sequence
        y.append(data[i+seq_length, target_column])  # Target is the next value in the target column
    return np.array(X), np.array(y)

seq_length = 2
target_column = 1  # We want to predict the next value of the 3rd column
X, y = create_sequences(data, seq_length, target_column)

# Define the model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=200, verbose=0)

# Predict the next value for a new sequence
test_sequence = np.array(df_test.loc[2527:2528])
test_sequence = test_sequence.reshape((1, seq_length, test_sequence.shape[1]))

predicted_value = model.predict(test_sequence)
print(f"Predicted next value for the 2rd column: {predicted_value[0][0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
Predicted next value for the 2rd column: -3.945070266723633


In [ ]:
# Predict the next value for a new sequence
test_sequence = np.array(df_test.loc[2505:2506])
test_sequence = test_sequence.reshape((1, seq_length, test_sequence.shape[1]))

predicted_value = model.predict(test_sequence)
print(f"Predicted next value for the 2rd column: {predicted_value[0][0]}")

In [40]:
df_test[2528:2529]

,Cycle Cumulé,Valeur suivante,Valeur flottante,Cycle greater or less,Valeur greater or less,Cycle_Moving Average,Cycle_Cumulative Deviation,Valeur_Moving Average,Valeur_Cumulative Deviation


In [ ]:
scaler=StandardScaler()
scaler.fit(x_train_new)
x_train_new=pd.DataFrame(scaler.transform(x_train_new),columns=x_train_new.columns,index=x_train_new.index)
x_test_new=pd.DataFrame(scaler.transform(x_test_new),columns=x_test_new.columns,index=x_test_new.index)
final_test_set=pd.DataFrame(scaler.transform(final_test_set),columns=final_test_set.columns,index=final_test_set.index)

In [ ]:
final_test_set

# Result

In [ ]:
model_new = RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=100)
model_new.fit(x_train_new,y_train_new)
model_new.score(x_test_new,y_test_new)

In [ ]:
y_test_new

In [ ]:
y_pred=model_new.predict(x_test_new)

# Root Mean Square Error

In [ ]:
np.sqrt(np.mean((y_pred-y_test_new)**2))

# XGBOOST REGRESSOR

In [ ]:
y_pred = model.predict(x_test_new)

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test_new, y_pred, squared=False)
print(f'RMSE: {rmse}')


In [ ]:
model.score(x_test_new,y_test_new)

In [ ]:
model.predict(final_test_set)

# Actual Value is 15 and predicted value is 16.43

In [ ]:
model.predict(x_test_new)

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.figure(figsize=(20, 6))

# Plot the actual values
plt.plot(y_test_new.reset_index(drop=True), label='Actual Values', color='blue', linestyle='-', marker='o')

# Plot the predicted values
plt.plot(y_pred, label='Predicted Values', color='red', linestyle='--', marker='x')

# Add labels and title
plt.xlabel('Time Steps')
plt.ylabel('Values')
plt.title('Actual vs Predicted Values')

# Show legend to distinguish between actual and predicted lines
plt.legend()

# Display the plot
plt.show()


In [ ]:
x_train_new.shape

# NEURAL NETWORK

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Example data (5 columns)
data = np.array([[0.1, 0.2, 0.3, 0.4, 0.5],
                 [0.6, 0.7, 0.8, 0.9, 1.0],
                 [1.1, 1.2, 1.3, 1.4, 1.5],
                 [1.6, 1.7, 1.8, 1.9, 2.0]])

# Prepare sequences (windowed sequences of input data)
def create_sequences(data, seq_length, target_column):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Use all columns in the sequence
        y.append(data[i+seq_length, target_column])  # Target is the next value in the target column
    return np.array(X), np.array(y)

seq_length = 2
target_column = 2  # We want to predict the next value of the 3rd column
X, y = create_sequences(data, seq_length, target_column)

# Define the model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=200, verbose=0)

# Predict the next value for a new sequence
test_sequence = np.array([[1.1, 1.2, 1.3, 1.4, 1.5],
                          [1.6, 1.7, 1.8, 1.9, 2.0]])
test_sequence = test_sequence.reshape((1, seq_length, test_sequence.shape[1]))

predicted_value = model.predict(test_sequence)
print(f"Predicted next value for the 3rd column: {predicted_value[0][0]}")


In [ ]:
tf_model.evaluate(x_test_new,y_test_new)

In [ ]:
y_pred = model.predict(x_test_new)
